In [ ]:
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
import chromadb

p = pipeline(
    task = Tasks.document_segmentation,
    model = 'iic/nlp_bert_document-segmentation_chinese-base'
)
jinaAI_base_embedding_model = HuggingFaceEmbedding(model_name='jinaai/jina-embeddings-v2-base-zh', pooling='mean', trust_remote_code=True)
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("bible_vector")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=None, embed_model=jinaAI_base_embedding_model)

In [1]:
import json
file_path = './new_bible.json'

# 打开并读取 JSON 文件
with open(file_path, 'r', encoding='utf-8') as file:
    # 加载 JSON 文件内容到一个字典
    data = json.load(file)

all_nodes = []

for key in data.keys():
    for i in len(data[key]):
        result = p(documents=data[key][i])
        text_list = [text.replace('\t', '') for text in result['text'].split('\n\t') if len(text.strip()) > 0]
        documents = [Document(text=t) for t in text_list]
        parser = SentenceSplitter()
        nodes = parser.get_nodes_from_documents(documents)
        for node in nodes:
            node.metadata["book"] = key
            node.metadata["chapter"] = i
        all_nodes.append(nodes)
vector_index = VectorStoreIndex(all_nodes, service_context=service_context, storage_context=storage_context)

创世记
出埃及记
利未记
民数记
申命记
约书亚记
士师记
路得记
撒母耳记上
撒母耳记下
列王纪上
列王纪下
历代志上
历代志下
以斯拉记
尼希米记
以斯帖记
约伯记
诗篇
箴言
传道书
雅歌
以赛亚书
耶利米书
耶利米哀歌
以西结书
但以理书
何西阿书
约珥书
阿摩司书
俄巴底亚书
约拿书
弥迦书
那鸿书
哈巴谷书
西番雅书
哈该书
撒迦利亚书
玛拉基书
马太福音
马可福音
路加福音
约翰福音
使徒行传
罗马书
哥林多前书
哥林多后书
加拉太书
以弗所书
腓立比书
歌罗西书
帖撒罗尼迦前书
帖撒罗尼迦后书
提摩太前书
提摩太后书
提多书
腓利门书
希伯来书
雅各书
彼得前书
彼得后书
约翰一书
约翰二书
约翰三书
犹大书
启示录
